## Instagram 스크랩


## 필요한 라이브러리 설치
* 아나콘다 사용시 다음의 프롬프트 창을 열어 conda 명령어로 설치합니다.
* pip 사용시 아래에 있는 명령어를 터미널로 설치합니다.
<img src="https://i.imgur.com/Sar4gdw.jpg">

### Selenium
* `conda install -c anaconda selenium`
* [Selenium :: Anaconda Cloud](https://anaconda.org/anaconda/selenium)

* pip 사용시 : `pip install selenium`

### BeautifulSoup
* `conda install -c anaconda beautifulsoup4`
* [Beautifulsoup4 :: Anaconda Cloud](https://anaconda.org/anaconda/beautifulsoup4)

* pip 사용시 : `pip install beautifulsoup4`

### tqdm
* `conda install -c conda-forge tqdm`
* [tqdm/tqdm: A Fast, Extensible Progress Bar for Python and CLI](https://github.com/tqdm/tqdm)
* `pip install tqdm`

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd
from pandas import read_excel
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import datetime as dt
import time
from tqdm import trange
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# 브라우저 기동
driver = webdriver.Chrome(options=options)
#driver = webdriver.Firefox()

# 인스타그램 로그인
id = 'inhwan.jung@gmail.com'
pw = '2020@Alpha'
phone = '821020843388'

# 클립보드에 input을 복사한 뒤
# 해당 내용을 actionChain을 이용해 로그인 폼에 붙여넣기
driver.get('https://www.instagram.com/accounts/login')
# 모든 동작마다 크롬브라우저가 준비될 때 까지 최대 3초씩 대기
driver.implicitly_wait(3)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(id)
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(pw)
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

In [2]:
# 핵심단어 읽어 오기
my_sheet = '소비키워드'
keywords_filename = 'deskresearch.xlsx'
df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
keywords = df['핵심단어'].values.tolist()
print(keywords)

# "1인 외식" 을 instagram 에서 검색 못하게 해서 제거하고 돌림
keywords = keywords[1:]

['1인 외식', '가치소비', 'RMR', '웰빙', '체험소비', '골목상권', '공유경제', '구독서비스', '로컬푸드', '모디슈머', '1인 미디어', '배달음식', '푸드홀', '스내킹 스토어', '스몰 럭셔리', '언택트', '에스닉푸드', '이색 식재료', '뉴밀리어', '지속가능', '친환경', '취향의 세분화', '콘텐츠', '콜라보레이션', '탈도심화 ', '특수부위', '펫', '푸드테크', 'K-푸드', '홈코노미', 'MZ세대', '럭셔리 미식', '듀얼매장', '무알콜', '안심식당']


In [3]:
keywords
keywords = keywords[4:]

In [ ]:
#get old tweets
# filename_list = []
# import csv

interval = 2

for keyword in keywords:
    
    url = f"https://www.instagram.com/explore/tags/{keyword}"
    driver.get(url)
    time.sleep(3)
    
    # 이미지 목록을 저장할 빈 리스트
    links = []
    link = ""
    
    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 지정된 회차 동안 반복하면서 스크롤을 화면 맨 아래로 이동한다.
    for i in trange(0, 150):

        # 수집 과정을 출력한다.
#         print("%04d번째 페이지에서 %02d건 수집함 >> 누적 데이터수: %05d" % (i+1, len(link), len(links)))
        current_len = len(links)

        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
        
        # 페이지 로딩 대기
        time.sleep(interval)  

        # 현재 브라우저에 표시되고 있는 소스코드 가져오기
        soup = bs(driver.page_source, 'html.parser')
        
        # srcset이라는 속성을 포함하는 모든 이미지 태그 가져오기 --> 리스트형으로 반환됨
        link = soup.select("a[href]")

        # 미리 준비한 리스트에 결합시킴
        links += link

        # 동일한 항목에 대한 중복제거
        links = list(set(links))
        
        # 현재 문서 뫂이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break
        
        if current_len == len(links):
            break
        
        # 다음 컨텐츠가 로딩되는 동안 1초씩 대기
        time.sleep(1)
    
    link_dict = { "link": links }
    link_df = pd.DataFrame(link_dict)
    # save to csv
    filename = "./urls/" + "insta-scrapped_" + keyword.replace(" ","") + ".csv"   
    link_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

  2%|█▍                                                                     | 3/150 [00:09<07:42,  3.14s/it]